<a href="https://colab.research.google.com/github/Ricardobigdata/PrimerRepor/blob/main/C%C3%B3pia_de_auditoria_cap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. conectar ao google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pasta_raiz = '/content/drive/MyDrive/AUDITORIA/2023'

## 1. importar bibliotecas

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas import DataFrame
from datetime import datetime
import pandas as pd
import numpy as np
import time
import warnings


## 2. importar tabelas

In [33]:
movimentos = pd.read_csv(f'{pasta_raiz}/ValoresMovimentoTitulos.txt', sep=';', encoding='utf-8')
movimentos.astype(str)
movimentos["valor"] = [x.replace(",", ".") for x in movimentos["vdrValor"]]
movimentos["movimento"] = movimentos["valor"].astype(float, errors = 'raise')
movimentos["mrfMesAno2"] = pd.to_datetime(movimentos['mrfMesAno'])
movimentos.fillna(0, inplace = False)

,entCodigo,mrfMesAno,plnCodigo,cmpID,vmtInicioVigTitulo,vdrValor,valor,movimento,mrfMesAno2
0,21491,1/12/2013 00:00:00,2,12800,1/10/2013 00:00:00,"0,00",0.00,0.0,2013-01-12
1,21491,1/12/2013 00:00:00,2,12800,1/11/2013 00:00:00,"3519,00",3519.00,3519.0,2013-01-12
2,21491,1/12/2013 00:00:00,2,12800,1/12/2013 00:00:00,"2944,00",2944.00,2944.0,2013-01-12
3,21491,1/12/2013 00:00:00,2,12801,1/10/2013 00:00:00,"0,00",0.00,0.0,2013-01-12
4,21491,1/12/2013 00:00:00,2,12801,1/11/2013 00:00:00,"0,00",0.00,0.0,2013-01-12
...,...,...,...,...,...,...,...,...,...
1254837,21491,1/12/2022 00:00:00,1718,12827,1/12/2022 00:00:00,"0,00",0.00,0.0,2022-01-12
1254838,21491,1/12/2022 00:00:00,1718,12828,1/12/2022 00:00:00,"0,00",0.00,0.0,2022-01-12
1254839,21491,1/12/2022 00:00:00,1718,12829,1/12/2022 00:00:00,"0,00",0.00,0.0,2022-01-12
1254840,21491,1/12/2022 00:00:00,1718,12830,1/12/2022 00:00:00,"36349,00",36349.00,36349.0,2022-01-12


In [34]:
provisao = pd.read_csv(f'{pasta_raiz}/ValoresProvTit.txt', sep=';', encoding='utf-8')
provisao.astype(str)
provisao["valor"] = [x.replace(".", "") for x in provisao['vdrValor']]
provisao["valor1"] = [x.replace(",", ".") for x in provisao["valor"]]
provisao["provisao"] = provisao["valor1"].astype(float, errors = 'raise')
provisao["mrfMesAno2"] = pd.to_datetime(provisao['mrfMesAno'])
provisao.fillna(0, inplace = False)


,entCodigo,mrfMesAno,plnCodigo,cmpID,vdrValor,valor,valor1,provisao,mrfMesAno2
0,21491,1/12/2011 00:00:00,1,2560,"20009,99","20009,99",20009.99,20009.99,2011-01-12
1,21491,1/12/2011 00:00:00,1,2561,"5,00","5,00",5.00,5.00,2011-01-12
2,21491,1/12/2011 00:00:00,1,2562,"1000,00","1000,00",1000.00,1000.00,2011-01-12
3,21491,1/12/2011 00:00:00,1,2563,"0,00","0,00",0.00,0.00,2011-01-12
4,21491,1/12/2011 00:00:00,1,2564,"0,00","0,00",0.00,0.00,2011-01-12
...,...,...,...,...,...,...,...,...,...
556892,21491,1/12/2022 00:00:00,1718,12791,"29079,20","29079,20",29079.20,29079.20,2022-01-12
556893,21491,1/12/2022 00:00:00,1718,12792,"0,00","0,00",0.00,0.00,2022-01-12
556894,21491,1/12/2022 00:00:00,1718,12793,"0,00","0,00",0.00,0.00,2022-01-12
556895,21491,1/12/2022 00:00:00,1718,12794,"4003,46","4003,46",4003.46,4003.46,2022-01-12


In [35]:
resumo_mov = movimentos.groupby(['plnCodigo','mrfMesAno2','cmpID'])['movimento'].sum()
resumo_prov = provisao.groupby(['plnCodigo','mrfMesAno2','cmpID'])['provisao'].sum()
resumo_mov = pd.DataFrame(resumo_mov).reset_index()
resumo_prov = pd.DataFrame(resumo_prov).reset_index()

In [36]:
empilhado_prov_tot = resumo_prov.query('mrfMesAno2 >= "2021-01-12" ' )
empilhado_mov_tot = resumo_mov.query('mrfMesAno2 >= "2021-01-12" ' )

## 3. Geração dos valores de provisões

In [37]:
PMC = empilhado_prov_tot.query('cmpID == 12781')
PDB = empilhado_prov_tot.query('cmpID == 12782')
PRA = empilhado_prov_tot.query('cmpID == 12785')
PRV = empilhado_prov_tot.query('cmpID == 12784')
PSR = empilhado_prov_tot.query('cmpID == 12786')
PCS = empilhado_prov_tot.query('cmpID == 12787')
PSRC = empilhado_prov_tot.query('cmpID == 12791')
PSP = empilhado_prov_tot.query('cmpID == 12793')
PDA = empilhado_prov_tot.query('cmpID == 12794')
OPT = empilhado_prov_tot.query('cmpID == 12795')

OPT = OPT.rename(columns={'provisao': 'OPT'})
PMC = PMC.rename(columns={'provisao': 'PMC'})
PRA = PRA.rename(columns={'provisao': 'PRA'})
PDB = PDB.rename(columns={'provisao': 'PDB'})
PRV = PRV.rename(columns={'provisao': 'PRV'})
PSR = PSR.rename(columns={'provisao': 'PSR'})
PCS = PCS.rename(columns={'provisao': 'PCS'})
PSRC = PSRC.rename(columns={'provisao': 'PSRC'})
PSP = PSP.rename(columns={'provisao': 'PSP'})
PDA = PDA.rename(columns={'provisao': 'PDA'})

## 4. Geração dos valores de movimentos das provisões

In [38]:
ARREC_INST = empilhado_mov_tot.query('cmpID == 12788')
ARREC_INST = ARREC_INST.rename(columns={'movimento': 'ARREC_INST'})
SORTEIO_REALIZADOS_INST = empilhado_mov_tot.query('cmpID == 12789')
SORTEIO_REALIZADOS_INST = SORTEIO_REALIZADOS_INST.rename(columns={'movimento': 'SORTEIO_REALIZADOS_INST'})
SORTEIO_A_REALIZAR_C_OBRIG = empilhado_mov_tot.query('cmpID == 12790')
SORTEIO_A_REALIZAR_C_OBRIG = SORTEIO_A_REALIZAR_C_OBRIG.rename(columns={'movimento': 'SORTEIO_A_REALIZAR_C_OBRIG'})
ARREC_N_CAP = empilhado_mov_tot.query('cmpID == 12808')
ARREC_N_CAP = ARREC_N_CAP.rename(columns={'movimento': 'ARREC_N_CAP'})
ARREC_CAP = empilhado_mov_tot.query('cmpID == 12809')
ARREC_CAP = ARREC_CAP.rename(columns={'movimento': 'ARREC_CAP'})
TRANSF_PMC_P_PR = empilhado_mov_tot.query('cmpID == 12810')
TRANSF_PMC_P_PR = TRANSF_PMC_P_PR.rename(columns={'movimento': 'TRANSF_PMC_P_PR'})
AJUST_PMC = empilhado_mov_tot.query('cmpID == 12811')
AJUST_PMC = AJUST_PMC.rename(columns={'movimento': 'AJUST_PMC'})
PENALT = empilhado_mov_tot.query('cmpID == 12812')
PENALT = PENALT.rename(columns={'movimento': 'PENALT'})
ARREC_BONUS = empilhado_mov_tot.query('cmpID == 12813')
ARREC_BONUS = ARREC_BONUS.rename(columns={'movimento': 'ARREC_BONUS'})
TRANSF_PDB_P_PR = empilhado_mov_tot.query('cmpID == 12814')
TRANSF_PDB_P_PR = TRANSF_PDB_P_PR.rename(columns={'movimento': 'TRANSF_PDB_P_PR'})
AJUST_PDB = empilhado_mov_tot.query('cmpID == 12815')
AJUST_PDB = AJUST_PDB.rename(columns={'movimento': 'AJUST_PDB'})
RESGATADO = empilhado_mov_tot.query('cmpID == 12816')
RESGATADO = RESGATADO.rename(columns={'movimento': 'RESGATADO'})
PRESCRICAO = empilhado_mov_tot.query('cmpID == 12817')
PRESCRICAO = PRESCRICAO.rename(columns={'movimento': 'PRESCRICAO'})
AJUST_PR = empilhado_mov_tot.query('cmpID == 12818')
AJUST_PR = AJUST_PR.rename(columns={'movimento': 'AJUST_PR'})
DEV_CUST_SORT_FUTURO = empilhado_mov_tot.query('cmpID == 12819')
DEV_CUST_SORT_FUTURO = DEV_CUST_SORT_FUTURO.rename(columns={'movimento': 'DEV_CUST_SORT_FUTURO'})
ARREC_SORT = empilhado_mov_tot.query('cmpID == 12820')
ARREC_SORT = ARREC_SORT.rename(columns={'movimento': 'ARREC_SORT'})
BAIXA_PSR = empilhado_mov_tot.query('cmpID == 12821')
BAIXA_PSR = BAIXA_PSR.rename(columns={'movimento': 'BAIXA_PSR'})
SORT_REALIZADO = empilhado_mov_tot.query('cmpID == 12822')
SORT_REALIZADO = SORT_REALIZADO.rename(columns={'movimento': 'SORT_REALIZADO'})
SORT_PAGOS = empilhado_mov_tot.query('cmpID == 12823')
SORT_PAGOS = SORT_PAGOS.rename(columns={'movimento': 'SORT_PAGOS'})
AJUST_PSP_PRESCRICAO = empilhado_mov_tot.query('cmpID == 12824')
AJUST_PSP_PRESCRICAO = AJUST_PSP_PRESCRICAO.rename(columns={'movimento': 'AJUST_PSP_PRESCRICAO'})
AJUST_PSP_OUTROS = empilhado_mov_tot.query('cmpID == 12825')
AJUST_PSP_OUTROS = AJUST_PSP_OUTROS.rename(columns={'movimento': 'AJUST_PSP_OUTROS'})

## 4.1. Construção da validação da PMC

In [ ]:


ARREC = PMC.merge(ARREC_CAP[['ARREC_CAP','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
TRANS = ARREC.merge(TRANSF_PMC_P_PR[['TRANSF_PMC_P_PR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
ajust = TRANS.merge(AJUST_PMC[['AJUST_PMC','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PMC = ajust.merge(PENALT[['PENALT','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PMC = pd.DataFrame(MOV_PMC).reset_index()
MOV_PMC.fillna(0, inplace = False)

In [40]:
MOV_PMC['plnCodigo1'] = MOV_PMC["plnCodigo"].shift(1)
MOV_PMC.fillna(0)
MOV_PMC['plnCodigo2'] = MOV_PMC['plnCodigo1'].astype('Int64')
MOV_PMC.fillna(0).all()
MOV_PMC['mrfMesAno_1'] = MOV_PMC["PMC"].shift(1)

MOV_PMC.loc[MOV_PMC['mrfMesAno2'] != '2021-01-12', 'PMC_1'] = MOV_PMC['mrfMesAno_1']
MOV_PMC.loc[MOV_PMC['plnCodigo2'] != MOV_PMC['plnCodigo'], 'PMC_1'] = 0 
MOV_PMC.loc[MOV_PMC['mrfMesAno2'] == '2021-01-12' , 'PMC_1'] = 0   

In [41]:
juros = 0.0016
tr={'mrfMesAno': 
    ['2021-01-12','2022-01-01','2022-01-02','2022-01-03','2022-01-04','2022-01-05','2022-01-06','2022-01-07','2022-01-08','2022-01-09','2022-01-10','2022-01-11','2022-01-12'], 
'tr': [0,0.000605,0,0.000971,0.000555,0.001663,0.001484,0.001631,0.002409,0.00180,0.001494,0.001507,0.002072]
}
tr_ = pd.DataFrame(data=tr)
tr_["mrfMesAno2"] = pd.to_datetime(tr_['mrfMesAno'])


In [42]:
MOV_PMC = MOV_PMC.merge(tr_, how='left', on="mrfMesAno2")

In [54]:
MOV_PMC['ARREC_CAP'] = pd.to_numeric(MOV_PMC['ARREC_CAP'], downcast='float', errors='coerce')
MOV_PMC['TRANSF_PMC_P_PR'] = pd.to_numeric(MOV_PMC['TRANSF_PMC_P_PR'], downcast='float', errors='coerce')
MOV_PMC['AJUST_PMC'] = pd.to_numeric(MOV_PMC['AJUST_PMC'], downcast='float', errors='coerce')
MOV_PMC['PENALT'] = pd.to_numeric(MOV_PMC['PENALT'], downcast='float', errors='coerce')
MOV_PMC['PMC_1'] = pd.to_numeric(MOV_PMC['PMC_1'], downcast='float', errors='coerce')
MOV_PMC['PMC'] = pd.to_numeric(MOV_PMC['PMC'], downcast='float', errors='coerce')
pd.to_datetime(MOV_PMC['mrfMesAno'])



fator1 = (1+float(f'{juros}'))*(1+MOV_PMC['tr'])
fator2 = ((1+float(f'{juros}'))**0.5)*(1+MOV_PMC['tr'])**0.5

MOV_PMC['calculo']  = (MOV_PMC['PMC_1']*fator1) + ((MOV_PMC['ARREC_CAP'] - MOV_PMC['TRANSF_PMC_P_PR']  - MOV_PMC['AJUST_PMC'] - MOV_PMC['PENALT'])*fator2)

MOV_PMC['controle'] = (MOV_PMC.PMC+MOV_PMC.ARREC_CAP+MOV_PMC.TRANSF_PMC_P_PR+MOV_PMC.AJUST_PMC+MOV_PMC.PENALT+MOV_PMC.PMC_1)

MOV_PMC['variacao'] = ((MOV_PMC.PMC-MOV_PMC.calculo)/MOV_PMC.PMC)*100

MOV_PMC_final = MOV_PMC[['plnCodigo','mrfMesAno','mrfMesAno2','PMC','ARREC_CAP','TRANSF_PMC_P_PR','AJUST_PMC','PENALT','PMC_1','calculo','variacao','controle']]

final = MOV_PMC_final[['mrfMesAno2','PMC','ARREC_CAP','TRANSF_PMC_P_PR','AJUST_PMC','PENALT','PMC_1','calculo']]
final.fillna(0)
final2 = final.groupby(by=["mrfMesAno2"]).sum().reset_index()
final2['calculo']  = (final2['PMC_1']*fator1) + ((final2['ARREC_CAP'] - final2['TRANSF_PMC_P_PR']  - final2['AJUST_PMC'] - final2['PENALT'])*fator2)
final2['variacao'] = ((final2['PMC']-final2['calculo'])/final2['PMC'])*100
def formatar(valor):
    return "{:,.2f}".format(valor)
final2['calculo'] = final2['calculo'].apply(formatar)
final2['PMC_1'] = final2['PMC_1'].apply(formatar)
final2['ARREC_CAP'] = final2['ARREC_CAP'].apply(formatar)
final2['TRANSF_PMC_P_PR'] = final2['TRANSF_PMC_P_PR'].apply(formatar)
final2['AJUST_PMC'] = final2['AJUST_PMC'].apply(formatar)
final2['PENALT'] = final2['PENALT'].apply(formatar)
final2['PMC'] = final2['PMC'].apply(formatar)
final2['variacao'] = final2['variacao'].apply(formatar)


In [56]:
final2.head(15)

,mrfMesAno2,PMC,ARREC_CAP,TRANSF_PMC_P_PR,AJUST_PMC,PENALT,PMC_1,calculo,variacao
0,2021-01-12,"80,818,040.00","52,615,080.00","47,626,028.00",0.00,0.00,0.00,"4,993,041.65",93.82
1,2022-01-01,"77,190,448.00","34,788,232.00","38,603,800.00",0.00,0.00,"80,818,040.00","77,176,547.82",0.02
2,2022-01-02,"65,653,452.00","34,407,312.00","46,081,696.00",0.00,0.00,"77,190,448.00","65,630,232.94",0.04
3,2022-01-03,"66,585,732.00","41,199,100.00","40,452,080.00",0.00,0.00,"65,653,452.00","66,570,329.28",0.02
4,2022-01-04,"67,465,720.00","33,289,502.00","32,555,412.00",0.00,0.00,"66,585,732.00","67,464,164.26",0.00
5,2022-01-05,"73,420,896.00","40,424,224.00","34,688,504.00",0.00,0.00,"67,465,720.00","73,431,117.98",-0.01
6,2022-01-06,"83,701,608.00","43,159,604.00","33,133,682.00",0.00,0.00,"73,420,896.00","83,688,882.33",0.02
7,2022-01-07,"72,059,424.00","37,347,888.00","49,260,284.00",0.00,0.00,"83,701,608.00","72,040,625.85",0.03
8,2022-01-08,"87,551,008.00","50,705,384.00","35,539,296.00",0.00,0.00,"72,059,424.00","87,545,075.16",0.01
9,2022-01-09,"86,598,920.00","35,673,368.00","36,977,428.00",0.00,0.00,"87,551,008.00","86,542,656.68",0.06



## 4.2. Construção da validação da PR

In [59]:


PR_TRAN = PRA.merge(TRANSF_PMC_P_PR[['TRANSF_PMC_P_PR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
TRANSF_ = PR_TRAN.merge(TRANSF_PDB_P_PR[['TRANSF_PDB_P_PR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
DEV_CUSTO = TRANSF_.merge(DEV_CUST_SORT_FUTURO[['DEV_CUST_SORT_FUTURO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
RESGATADO = DEV_CUSTO.merge(RESGATADO[['RESGATADO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
PRESCRICAO = RESGATADO.merge(PRESCRICAO[['PRESCRICAO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PR = PRESCRICAO.merge(AJUST_PR[['AJUST_PR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PR2 = MOV_PR.groupby(by=["mrfMesAno2"]).sum().reset_index()

MOV_PR2['PRA_1'] = MOV_PR2["PRA"].shift(1)

MOV_PR2['calculo']  = (MOV_PR2['PRA_1']*fator1) + ((MOV_PR2['TRANSF_PMC_P_PR'] + MOV_PR2['TRANSF_PDB_P_PR']  - MOV_PR2['DEV_CUST_SORT_FUTURO'] - MOV_PR2['RESGATADO']-MOV_PR2['PRESCRICAO']- MOV_PR2['AJUST_PR'])*fator2)

MOV_PR2['variacao'] = ((MOV_PR2['PRA']-MOV_PR2['calculo'])/MOV_PR2['PRA'])*100

MOV_PR2['TRANSF_PMC_P_PR'] = MOV_PR2['TRANSF_PMC_P_PR'].apply(formatar)
MOV_PR2['TRANSF_PDB_P_PR'] = MOV_PR2['TRANSF_PDB_P_PR'].apply(formatar)
MOV_PR2['DEV_CUST_SORT_FUTURO'] = MOV_PR2['DEV_CUST_SORT_FUTURO'].apply(formatar)
MOV_PR2['RESGATADO'] = MOV_PR2['RESGATADO'].apply(formatar)
MOV_PR2['PRESCRICAO'] = MOV_PR2['PRESCRICAO'].apply(formatar)
MOV_PR2['AJUST_PR'] = MOV_PR2['AJUST_PR'].apply(formatar)
MOV_PR2['PRA'] = MOV_PR2['PRA'].apply(formatar)
MOV_PR2['PRA_1'] = MOV_PR2['PRA_1'].apply(formatar)
MOV_PR2['calculo'] = MOV_PR2['calculo'].apply(formatar)
MOV_PR2['variacao'] = MOV_PR2['variacao'].apply(formatar)

MOV_PR2.head(15)


,mrfMesAno2,plnCodigo,cmpID,PRA,TRANSF_PMC_P_PR,TRANSF_PDB_P_PR,DEV_CUST_SORT_FUTURO,RESGATADO,PRESCRICAO,AJUST_PR,PRA_1,calculo,variacao
0,2021-01-12,734900,15329215,"10,884,212.37","47,626,027.55",0.00,0.00,"36,787,533.18",0.00,0.00,nan,nan,nan
1,2022-01-01,775457,15751120,"102,718.94","38,603,797.92",0.00,0.00,"49,385,310.19",0.00,0.00,"10,884,212.37","114,825.04",-11.79
2,2022-01-02,820943,16211380,"8,475,181.49","46,081,697.14",0.00,0.00,"37,709,234.59",0.00,0.00,"102,718.94","8,482,041.13",-0.08
3,2022-01-03,869043,16684425,"245,350.31","40,452,079.04",0.00,0.00,"48,681,918.45",0.00,0.00,"8,475,181.49","256,565.89",-4.57
4,2022-01-04,910397,17080760,"31,632.25","32,555,411.27",0.00,0.00,"32,769,154.92",0.00,0.00,"245,350.31","31,905.33",-0.86
5,2022-01-05,967958,17617730,"223,690.30","34,688,504.01",0.00,0.00,"34,496,511.69",0.00,0.00,"31,632.25","224,041.11",-0.16
6,2022-01-06,1041668,18282550,"142,112.15","33,133,681.08",0.00,0.00,"33,215,308.95",0.00,0.00,"223,690.30","142,626.95",-0.36
7,2022-01-07,1075133,18576605,"213,805.66","49,260,284.10",0.00,0.00,"49,188,635.47",0.00,0.00,"142,112.15","214,336.06",-0.25
8,2022-01-08,1130078,19049650,"34,325.49","35,539,294.11",0.00,0.00,"35,718,846.61",0.00,0.00,"213,805.66","34,751.23",-1.24
9,2022-01-09,1167978,19369275,"383,847.07","36,977,426.17",0.00,0.00,"36,627,989.52",0.00,0.00,"34,325.49","384,472.99",-0.16




## 4.3. Construção da validação da PSR



In [61]:


ARREC_PSR = PSR.merge(ARREC_SORT[['ARREC_SORT','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PSR = ARREC_PSR.merge(BAIXA_PSR[['BAIXA_PSR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PSR2 = MOV_PSR.groupby(by=["mrfMesAno2"]).sum().reset_index()

MOV_PSR2['PSR_1'] = MOV_PSR2["PSR"].shift(1)

MOV_PSR2['calculo']  = (MOV_PSR2['PSR_1']*fator1) + ((MOV_PSR2['ARREC_SORT'] - MOV_PSR2['BAIXA_PSR'])*fator2)
MOV_PSR2['variacao'] = ((MOV_PSR2['PSR']-MOV_PSR2['calculo'])/MOV_PSR2['PSR'])*100

MOV_PSR2['PSR'] = MOV_PSR2['PSR'].apply(formatar)
MOV_PSR2['ARREC_SORT'] = MOV_PSR2['ARREC_SORT'].apply(formatar)
MOV_PSR2['BAIXA_PSR'] = MOV_PSR2['BAIXA_PSR'].apply(formatar)
MOV_PSR2['calculo'] = MOV_PSR2['calculo'].apply(formatar)
MOV_PSR2['variacao'] = MOV_PSR2['variacao'].apply(formatar)
MOV_PSR2.head(15)


,mrfMesAno2,plnCodigo,cmpID,PSR,ARREC_SORT,BAIXA_PSR,PSR_1,calculo,variacao
0,2021-01-12,734900,15330414,"1,760,213.94","16,736,368.29","15,202,041.43",NaN,nan,nan
1,2022-01-01,775457,15752352,"204,897.29","11,063,823.59","12,620,128.01",1760213.94,"206,076.86",-0.58
2,2022-01-02,820943,16212648,"175,845.00","10,703,207.15","10,732,288.59",204897.29,"176,120.43",-0.16
3,2022-01-03,869043,16685730,"2,594,932.68","13,460,272.15","11,041,562.10",175845.00,"2,598,116.55",-0.12
4,2022-01-04,910397,17082096,"2,567,135.28","11,010,117.49","11,038,624.05",2594932.68,"2,571,989.79",-0.19
5,2022-01-05,967958,17619108,"196,983.15","13,754,244.67","16,125,014.11",2567135.28,"200,881.32",-1.98
6,2022-01-06,1041668,18283980,"2,748,059.00","13,925,926.42","11,375,379.44",196983.15,"2,752,071.03",-0.15
7,2022-01-07,1075133,18578058,"271,963.86","11,685,729.55","14,162,986.50",2748059.00,"275,686.19",-1.37
8,2022-01-08,1130078,19051140,"2,740,041.98","17,881,695.20","15,414,517.37",271963.86,"2,745,178.30",-0.19
9,2022-01-09,1167978,19370790,"2,793,312.72","12,055,023.97","12,002,465.11",2740041.98,"2,802,014.22",-0.31


## 4.4. Construção da validação da PSP

In [65]:


PSP_SORT= PSP.merge(SORT_REALIZADO[['SORT_REALIZADO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV = PSP_SORT.merge(SORT_PAGOS[['SORT_PAGOS','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV2 = MOV.merge(AJUST_PSP_PRESCRICAO[['AJUST_PSP_PRESCRICAO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PSP = MOV2.merge(AJUST_PSP_OUTROS[['AJUST_PSP_OUTROS','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PSP2 = MOV_PSP.groupby(by=["mrfMesAno2"]).sum().reset_index()

MOV_PSP2['PSP_1'] = MOV_PSP2["PSP"].shift(1)

MOV_PSP2['calculo']  = (MOV_PSP2['PSP_1']*fator1) + ((MOV_PSP2['SORT_REALIZADO'] - MOV_PSP2['SORT_PAGOS']-MOV_PSP2['AJUST_PSP_PRESCRICAO']-MOV_PSP2['AJUST_PSP_OUTROS'] )*fator2)
MOV_PSP2['variacao'] = ((MOV_PSP2['PSP']-MOV_PSP2['calculo'])/MOV_PSP2['PSP'])*100

MOV_PSP2['PSP'] = MOV_PSP2['PSP'].apply(formatar)
MOV_PSP2['PSP_1'] = MOV_PSP2['PSP_1'].apply(formatar)
MOV_PSP2['SORT_REALIZADO'] = MOV_PSP2['SORT_REALIZADO'].apply(formatar)
MOV_PSP2['SORT_PAGOS'] = MOV_PSP2['SORT_PAGOS'].apply(formatar)
MOV_PSP2['AJUST_PSP_PRESCRICAO'] = MOV_PSP2['AJUST_PSP_PRESCRICAO'].apply(formatar)
MOV_PSP2['AJUST_PSP_OUTROS'] = MOV_PSP2['AJUST_PSP_OUTROS'].apply(formatar)
MOV_PSP2['calculo'] = MOV_PSP2['calculo'].apply(formatar)
MOV_PSP2['variacao'] = MOV_PSP2['variacao'].apply(formatar)
MOV_PSP2.head(15)


,mrfMesAno2,plnCodigo,cmpID,PSP,SORT_REALIZADO,SORT_PAGOS,AJUST_PSP_PRESCRICAO,AJUST_PSP_OUTROS,PSP_1,calculo,variacao
0,2021-01-12,734900,15338807,"10,920,989.86","38,538,671.38","41,693,327.90",0.00,0.00,nan,nan,nan
1,2022-01-01,775457,15760976,"14,209,552.37","30,346,339.58","27,062,577.31",0.00,0.00,"10,920,989.86","14,232,463.43",-0.16
2,2022-01-02,820943,16221524,"13,670,377.36","25,963,290.11","26,502,465.12",0.00,0.00,"14,209,552.37","13,692,681.48",-0.16
3,2022-01-03,869043,16694865,"8,653,738.88","25,459,065.59","30,483,947.54",0.00,0.00,"13,670,377.36","8,674,203.95",-0.24
4,2022-01-04,910397,17091448,"8,129,065.73","26,346,482.97","26,875,825.35",0.00,0.00,"8,653,738.88","8,142,482.70",-0.17
5,2022-01-05,967958,17628754,"10,710,369.16","36,934,005.41","34,281,669.91","86,468.85",0.00,"8,129,065.73","10,725,665.36",-0.14
6,2022-01-06,1041668,18293990,"10,248,634.68","27,584,294.90","28,054,992.18","5,035.76",0.00,"10,710,369.16","10,266,958.75",-0.18
7,2022-01-07,1075133,18588229,"12,445,735.19","34,620,691.05","32,420,881.75","17,554.29",0.00,"10,248,634.68","12,467,555.21",-0.18
8,2022-01-08,1130078,19061570,"10,034,366.27","33,676,556.27","36,019,990.70","90,921.33",0.00,"12,445,735.19","10,056,442.89",-0.22
9,2022-01-09,1167978,19381395,"8,091,469.55","28,583,506.74","30,546,669.00","1,349.70",0.00,"10,034,366.27","8,100,660.39",-0.11


In [52]:
#MOV_PMC.to_csv("/content/drive/MyDrive/AUDITORIA/2023/MOV_PMC.csv")